In [ ]:
# Калініченко Назар Володимирович	Варіант 1

In [4]:
pip install pandas

SyntaxError: invalid syntax (601106015.py, line 1)

In [5]:
pip install db-dtypes

   ---------------------------------------- 0.0/24.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/24.8 MB 2.2 MB/s eta 0:00:12
   ---------------------------------------- 0.3/24.8 MB 4.0 MB/s eta 0:00:07
    --------------------------------------- 0.6/24.8 MB 4.4 MB/s eta 0:00:06
   - -------------------------------------- 0.8/24.8 MB 4.9 MB/s eta 0:00:05
   - -------------------------------------- 1.0/24.8 MB 4.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.3/24.8 MB 4.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.4/24.8 MB 4.6 MB/s eta 0:00:06
   -- ------------------------------------- 1.6/24.8 MB 4.3 MB/s eta 0:00:06
   -- ------------------------------------- 1.8/24.8 MB 4.1 MB/s eta 0:00:06
   --- ------------------------------------ 1.9/24.8 MB 4.0 MB/s eta 0:00:06
   --- ------------------------------------ 1.9/24.8 MB 3.9 MB/s eta 0:00:06
   --- ------------------------------------ 2.0/24.8 MB 3.6 MB/s eta 0:00:07
   ---

In [1]:
from google.cloud import bigquery

client = bigquery.Client(project='nk-cloud-course-iav-21')

In [12]:
# Завдання 1
# Для кожного дня, представленого в таблицях user_actions та courier_actions, розрахуйте такі показники:
# Число нових користувачів.
# Число нових кур’єрів.
# Загальна кількість користувачів на сьогодні.
# Загальна кількість кур’єрів на цей день.
# Колонки з показниками назвіть відповідно new_users, new_couriers, total_users, total_couriers. Колонку з датами назвіть date. Простежте, щоб показники були виражені цілими числами. Результат має бути відсортований за зростанням дати.
# Пояснення:

# Новими вважатимемо тих користувачів та кур’єрів, які в цей день здійснили свою першу дію в нашому сервісі. Загальна кількість користувачів/кур’єрів на поточний день — це результат додавання числа нових користувачів/кур’єрів у поточний день зі значеннями аналогічного показника всіх попередніх днів.
query_1 = """
WITH user_actions AS (
    SELECT
        DATE(time) AS date,
        user_id
    FROM
        `nk-cloud-course-iav-21.food_delivery.user_actions`
),
courier_actions AS (
    SELECT
        DATE(time) AS date,
        courier_id
    FROM
        `nk-cloud-course-iav-21.food_delivery.courier_actions`
),
users AS (
    SELECT
        date,
        COUNT(DISTINCT user_id) AS new_users
    FROM
        user_actions
    GROUP BY
        date
),
couriers AS (
    SELECT
        date,
        COUNT(DISTINCT courier_id) AS new_couriers
    FROM
        courier_actions
    GROUP BY
        date
),
total_users AS (
    SELECT
        date,
        SUM(new_users) OVER (ORDER BY date) AS total_users
    FROM
        users
),
total_couriers AS (
    SELECT
        date,
        SUM(new_couriers) OVER (ORDER BY date) AS total_couriers
    FROM
        couriers
),
result AS (
    SELECT
        users.date,
        new_users,
        new_couriers,
        total_users.total_users,
        total_couriers.total_couriers
    FROM
        users
    JOIN
        couriers
    ON
        users.date = couriers.date
    JOIN
        total_users
    ON
        users.date = total_users.date
    JOIN
        total_couriers
    ON
        users.date = total_couriers.date
)
SELECT
    *
FROM
    result
ORDER BY
    date
"""
query_job = client.query(query_1)
result_1 = query_job.result().to_dataframe()
result_1

,date,new_users,new_couriers,total_users,total_couriers
0,2022-08-24,134,95,134,95
1,2022-08-25,848,335,982,430
2,2022-08-26,1183,532,2165,962
3,2022-08-27,1604,727,3769,1689
4,2022-08-28,2145,937,5914,2626
5,2022-08-29,2289,1028,8203,3654
6,2022-08-30,2340,1148,10543,4802
7,2022-08-31,2536,1300,13079,6102
8,2022-09-01,2816,1491,15895,7593
9,2022-09-02,3547,1749,19442,9342


In [14]:
# Завдання 2
# Аналізуючи динаміку показників із попереднього завдання, ви могли помітити, що порівнювати абсолютні значення не дуже зручно. Давайте порахуємо динаміку показників у відносних величинах.

# Доповніть запит із попереднього завдання і тепер для кожного дня, поданого в таблицях user_actions та courier_actions, додатково розрахуйте такі показники:

# Приріст числа нових користувачів.
# Приріст числа нових кур’єрів.
# Приріст загальної кількості користувачів.
# Приріст загальної кількості кур’єрів.

# Показники, розраховані на попередньому кроці, також включіть у результуючу таблицю.

# Колонки з новими показниками назвіть відповідно new_users_change, new_couriers_change, total_users_growth, total_couriers_growth. Колонку з датами назвіть date.

# Усі показники приросту рахуйте у відсотках відносно попереднього дня. Під час розрахунку показників округляйте значення до двох знаків після коми.

# Результуюча таблиця має бути відсортована за зростанням дати.

# Поля в результуючій таблиці: date, new_users, new_couriers, total_users, total_couriers, new_users_change, new_couriers_change, total_users_growth, total_couriers_growth.

# Пояснення:

# Не забувайте при діленні заздалегідь наводити значення до потрібного типу даних. Пропущені значення приростів для першої дати не заповнюйте - просто залиште поля в цьому рядку порожніми.

query_2 = """
WITH user_actions AS (
    SELECT
        DATE(time) AS date,
        user_id
    FROM
        `nk-cloud-course-iav-21.food_delivery.user_actions`
),
courier_actions AS (
    SELECT
        DATE(time) AS date,
        courier_id
    FROM
        `nk-cloud-course-iav-21.food_delivery.courier_actions`
),
users AS (
    SELECT
        date,
        COUNT(DISTINCT user_id) AS new_users
    FROM
        user_actions
    GROUP BY
        date
),
couriers AS (
    SELECT
        date,
        COUNT(DISTINCT courier_id) AS new_couriers
    FROM
        courier_actions
    GROUP BY
        date
),
total_users AS (
    SELECT
        date,
        SUM(new_users) OVER (ORDER BY date) AS total_users
    FROM
        users
),
total_couriers AS (
    SELECT
        date,
        SUM(new_couriers) OVER (ORDER BY date) AS total_couriers
    FROM
        couriers
),
result AS (
    SELECT
        users.date,
        new_users,
        new_couriers,
        total_users.total_users,
        total_couriers.total_couriers
    FROM
        users
    JOIN
        couriers
    ON
        users.date = couriers.date
    JOIN
        total_users
    ON
        users.date = total_users.date
    JOIN
        total_couriers
    ON
        users.date = total_couriers.date
),
result_with_changes AS (
    SELECT
        result.date,
        new_users,
        new_couriers,
        total_users,
        total_couriers,
            ROUND(
                (new_users - LAG(new_users, 1) OVER (ORDER BY date)) / LAG(new_users, 1) OVER (ORDER BY date) * 100,
                2
            ) AS new_users_change,
            ROUND(
                (new_couriers - LAG(new_couriers, 1) OVER (ORDER BY date)) / LAG(new_couriers, 1) OVER (ORDER BY date) * 100,
                2
            ) AS new_couriers_change,
            ROUND(
                (total_users - LAG(total_users, 1) OVER (ORDER BY date)) / LAG(total_users, 1) OVER (ORDER BY date) * 100,
                2
            ) AS total_users_growth,
            ROUND(
                (total_couriers - LAG(total_couriers, 1) OVER (ORDER BY date)) / LAG(total_couriers, 1) OVER (ORDER BY date) * 100,
                2
            ) AS total_couriers_growth
    FROM
        result
)
SELECT
    *
FROM
    result_with_changes
ORDER BY
    date
"""
query_job = client.query(query_2)
result_2 = query_job.result().to_dataframe()
result_2

,date,new_users,new_couriers,total_users,total_couriers,new_users_change,new_couriers_change,total_users_growth,total_couriers_growth
0,2022-08-24,134,95,134,95,NaN,NaN,NaN,NaN
1,2022-08-25,848,335,982,430,532.84,252.63,632.84,352.63
2,2022-08-26,1183,532,2165,962,39.50,58.81,120.47,123.72
3,2022-08-27,1604,727,3769,1689,35.59,36.65,74.09,75.57
4,2022-08-28,2145,937,5914,2626,33.73,28.89,56.91,55.48
5,2022-08-29,2289,1028,8203,3654,6.71,9.71,38.70,39.15
6,2022-08-30,2340,1148,10543,4802,2.23,11.67,28.53,31.42
7,2022-08-31,2536,1300,13079,6102,8.38,13.24,24.05,27.07
8,2022-09-01,2816,1491,15895,7593,11.04,14.69,21.53,24.43
9,2022-09-02,3547,1749,19442,9342,25.96,17.30,22.32,23.03


In [15]:
# Завдання 3
# Збережіть отримані результати в окремі представлення в BigQuery.
# Підключіться до Looker Studio та побудуйте візуалізацію для отриманих даних.
# Збережіть створену візуалізацію в окремий дашборд.
# Поділіться посиланням на дашборд.
# Очікується, що для Завдання 1 буде використані лінійні графіки, а для Завдання 2 - стовпчасті діаграми.

# Але Ви можете запропонувати свої варіанти візуалізацій, якщо вони вважаєте, що вони краще підходять для відображення отриманих даних.

create_view_query_1 = f"CREATE OR REPLACE VIEW `nk-cloud-course-iav-21.food_delivery.user_actions_summary` AS {query_1}"
create_view_query_2 = f"CREATE OR REPLACE VIEW `nk-cloud-course-iav-21.food_delivery.user_actions_summary_with_changes` AS {query_2}"
client.query(create_view_query_1)
client.query(create_view_query_2)

QueryJob<project=nk-cloud-course-iav-21, location=EU, id=c5f0c420-36c5-4cdd-be8b-80d91c569140>

In [ ]:
# Посилання на дашборд:
https://lookerstudio.google.com/reporting/48aeed7b-9d80-41a9-833a-715b9e0b6359
https://lookerstudio.google.com/reporting/f226134e-e911-492a-9af0-1d7f5b1d2c09